# NeuroHub Tools and Tasks

## 0. Set up boilerplate

Make sure the optional package is installed then import the packages we'll use.

In [ ]:
!pip install requests

In [ ]:
import requests
import json

All API requests are done with respect to the `base_url` and with the API token from [NeuroHub New API Token](https://portal.neurohub.ca/nh_users/new_token).

In [ ]:
base_url = 'https://portal.cbrain.mcgill.ca'
cbrain_api_token = "PASTE_NEUROHUB_API_TOKEN_HERE"

## 1. Session

Do we have a valid session?

In [ ]:
session_response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if session_response.status_code != requests.codes.ok:
    print('Session is not valid.')
    print(json.dumps(session_response.json(), indent=4))
else:
    print('Session is valid.')
    session_info = session_response.json()
    user_id = str(session_info['user_id'])
    print("User ID:", user_id)

## 2. Tool Configs / Pipelines

List all possible pipelines.  That is, list all tool configurations where a particular tool can run on a particular bourreau.

In [ ]:
tool_configs = []
tool_configs_request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    tool_configs_response = requests.get(
        url = '/'.join([base_url, 'tool_configs']),
        data = tool_configs_request,
        headers = {'Accept': 'application/json'}
    )

    if tool_configs_response.status_code != requests.codes.ok:
        print('Tools request failed.')
        print(tool_configs_response)
        break

    tool_configs += tool_configs_response.json()
    tool_configs_request['page'] += 1
    
    if len(tool_configs_response.json()) < tool_configs_request['per_page']:
        break 
    
print("{} tool configs found\n".format(str(len(tool_configs))))
print(json.dumps(tool_configs, indent=4))

As we did before with user files, we can now filter the tools by words in their description. Let's filter to find all tools with the word `FSL` in the description. 

In [ ]:
tool_configs_filtered = list(filter(lambda f: 'FSL' in f['description'], tool_configs))
print(json.dumps(tool_configs_filtered, indent=4))

In [ ]:
tool_config_721 = list(filter(lambda f: f['id'] == 721, tool_configs))
print(json.dumps(tool_config_721, indent=4))

You can use the `tool_id` and `bourreau_id` to get information about these and verify this is the correct pipeline you wish to use.

In [ ]:
tools_response = requests.get(
    url = '/'.join([base_url, 'tools']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)
tool_51 = list(filter(lambda f: f['id'] == 51, tools_response.json()))
print(json.dumps(tool_51, indent=4))

In [ ]:
bourreaux_response = requests.get(
    url = '/'.join([base_url, 'bourreaux', str(39)]),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)
print(json.dumps(bourreaux_response.json(), indent=4))

## 3. Submit a task

Let's submit a task as ourselves (`user_id`) using the tool config we looked at above (`721`) with an example file as input and then output the results to MainStore.

In [ ]:
user_id = PASTE_YOUR_USER_ID_HERE
tool_config_id = 721
file_id = 3319815  # An example T1 weighted NIfTI file
data_provider_id = 27  # MainStore

task_headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}
task_params = (
    ('cbrain_api_token', cbrain_api_token),
)

# Specify the parameters of the task in a dictionary.
# We are running FSLFirst to register the brain scan.
task_data = {
  "cbrain_task": {
    'type': 'CbrainTask::FslFirst', 
    'user_id': user_id,
    'tool_config_id': tool_config_id,
    'params': { 
      # These parameters depend on the tool used
      'interface_userfile_ids': [file_id], 
      'input_file': file_id, 
      'prefix': 'output', 
      'brain_extracted': False, 
      'three_stage': False, 
      'verbose': False       
    }, 
    'run_number': None, 
    'results_data_provider_id': data_provider_id, 
    'cluster_workdir_size': None, 
    'workdir_archived': True, 
    'description': 'My test task!'}
}

task_response = requests.post(
    url = '/'.join([base_url, 'tasks']),
    headers = task_headers, 
    params = task_params,
    data = json.dumps(task_data)
)

if task_response.status_code == 200:
    task_info = task_response.json()
    print(json.dumps(task_info, indent=4))
else:
    print("Task posting failed.")
    print(task_response.text)

Let's track the completion of the task.

In [ ]:
taskID = task_info[0]["id"]

task_status = requests.get(
    url = '/'.join([base_url, 'tasks', str(taskID)]),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if task_status.status_code == 200:
    print(json.dumps(task_status.json(), indent=4))
else:
    print("Task status retrieval failed.")
    print(task_status.text)

## 4. Destroy the session

In [ ]:
session_destroy_response = requests.delete(
    url= '/'.join([base_url, 'session']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

if session_destroy_response.status_code == requests.codes.ok:
    print("Session ended.")
else:
    print("Session failure.")